In [1]:
import pandas as pd

from tqdm import tqdm
tqdm.pandas()

import gensim.downloader
import numpy as np

from pathlib import Path

import glob
import re

# Создание конечного датафрейма

In [2]:
# columns_data_path = "./collecting/columns_data.csv"
# if not Path(columns_data_path).exists():
#     raise Exception("columns_data.csv doesnt exist. Run `make; ./collect_columns_data` from this dir.")

# df_data = pd.read_csv(f"columns_data.csv", sep="⇧", engine="python", quotechar='"', on_bad_lines="warn")

In [3]:
import pandas as pd
import glob

df_list = []

number_of_chunks = len(glob.glob("../collecting/columns_data/data*.csv"))
for i in range(number_of_chunks):
    df_list.append(
        pd.read_csv(f"../collecting/columns_data/data_{i}.csv", sep="⇧", engine="python", quotechar='"', on_bad_lines="warn")
    )

df_data = pd.concat(df_list, axis=0)

Skipping line 708: '⇧' expected after '"'
Skipping line 6668: '⇧' expected after '"'
Skipping line 7141: '⇧' expected after '"'
Skipping line 8661: '⇧' expected after '"'
Skipping line 8863: field larger than field limit (131072)
Skipping line 9233: '⇧' expected after '"'
Skipping line 11144: '⇧' expected after '"'
Skipping line 11923: '⇧' expected after '"'
Skipping line 12338: '⇧' expected after '"'
Skipping line 13211: '⇧' expected after '"'
Skipping line 16581: '⇧' expected after '"'
Skipping line 16602: '⇧' expected after '"'
Skipping line 17339: '⇧' expected after '"'
Skipping line 17516: '⇧' expected after '"'
Skipping line 18088: '⇧' expected after '"'
Skipping line 20082: '⇧' expected after '"'
Skipping line 20495: '⇧' expected after '"'
Skipping line 20799: '⇧' expected after '"'
Skipping line 21002: '⇧' expected after '"'
Skipping line 23813: '⇧' expected after '"'
Skipping line 24086: '⇧' expected after '"'
Skipping line 24153: '⇧' expected after '"'
Skipping line 25834: '⇧

In [4]:
df_data_filter = df_data[~df_data["column_data"].isna()]
df_data_filter

,table_id,column_id,column_data
0,dataset/Dataset/data/1000006/table_0,0,Театр им. Евгения Вахтангова (Москва) Театр им...
1,dataset/Dataset/data/1000007/table_1,2,"Кулагин и партнёры Люба, дети и завод… Врачебн..."
2,dataset/Dataset/data/1000023/table_0,0,1997 2002 2003 2003 2006—2007 2008 2010 2010 2...
3,dataset/Dataset/data/1000078/table_0,2,смешная жизни(Таисия)Приезжая Летняя поездка к...
4,dataset/Dataset/data/1000247/table_0,2,Семейные страсти В пасти безумия Гаррисон Берж...
...,...,...,...
122940,dataset/Dataset/data/999585/table_0,2,Трек-лист [7] № Название Длительность 1. «Анге...
122941,dataset/Dataset/data/999746/table_3,20,НКЛ НКЛ
122942,dataset/Dataset/data/999874/table_0,5,5 из 9 8½ из 17 14 из 18 4½ из 15 14½ из 19 7 ...
122943,dataset/Dataset/data/999913/table_1,0,28-4-1 28-3-1 28-2-1 28-2 28-1 27-1 26-1 25-1 ...


## Очистка мета-информации

- Очищена мета-информация .mv-parser...
- Очищены ссылки типа ***some-text[1]***.

### .mw-parser

In [5]:
mv_parser = df_data_filter["column_data"].str.contains(".mw-parser")
df_data_filter[mv_parser]

,table_id,column_id,column_data
166,dataset/Dataset/data/1009955/table_1,5,22 февраля 1993.mw-parser-output .ts-comment-c...
171,dataset/Dataset/data/1010117/table_1,5,22 февраля 1993.mw-parser-output .ts-comment-c...
582,dataset/Dataset/data/1035911/table_1,2,.mw-parser-output .ts-comment-commentedText{bo...
3556,dataset/Dataset/data/1066312/table_0,1,1991 1992 1993 1994 1995 1996 1997 1998 2001 2...
5596,dataset/Dataset/data/12157/table_1,0,Истории Изгой-один Хан Соло Прочие спин-оффы Э...
...,...,...,...
121631,dataset/Dataset/data/919988/table_0,12,6-е место из 8 2-е место из 3 Переход на 3-й у...
121854,dataset/Dataset/data/932005/table_2,7,42/48либо40/44 42/48либо40/44 42/48либо40/44 4...
122085,dataset/Dataset/data/944485/table_2,1,«ЖАRA» «Чайный пьяница» «Рвы» GTA 4 (видеоигра...
122407,dataset/Dataset/data/970269/table_1,5,22 февраля 1993.mw-parser-output .ts-comment-c...


In [6]:
df_data_filter.loc[:, "column_data"] = df_data_filter["column_data"].progress_apply(
    lambda x: re.sub(r"\.mw-parser-output \.ts.+{.+}", "", x)
)
df_data_filter["column_data"].str.contains("mw-parser").sum()

100%|████████████████████████████| 1447089/1447089 [00:01<00:00, 1048462.61it/s]


0

In [7]:
# def clean_mv_parser(col_name: str) -> str:
#     if col_name.strip().startswith(".mw-parser"):
#         return col_name.split("}")[-1]
#     return col_name

# df_data_filter.loc[:, "column_data"] = df_data_filter["column_data"].progress_apply(lambda x: clean_mv_parser(str(x)))
# df_data_filter

In [8]:
df_data_filter[mv_parser]

,table_id,column_id,column_data
166,dataset/Dataset/data/1009955/table_1,5,22 февраля 199328 ноября 1997[13]1 декабря 199...
171,dataset/Dataset/data/1010117/table_1,5,22 февраля 199328 ноября 1997[13]1 декабря 199...
582,dataset/Dataset/data/1035911/table_1,2,22 декабря 2005[6]18 декабря 2020[7]
3556,dataset/Dataset/data/1066312/table_0,1,1991 1992 1993 1994 1995 1996 1997 1998 2001 2...
5596,dataset/Dataset/data/12157/table_1,0,Истории Изгой-один Хан Соло Прочие спин-оффы Э...
...,...,...,...
121631,dataset/Dataset/data/919988/table_0,12,6-е место из 8 2-е место из 3 Переход на 3-й у...
121854,dataset/Dataset/data/932005/table_2,7,42/48либо40/44 42/48либо40/44 42/48либо40/44 4...
122085,dataset/Dataset/data/944485/table_2,1,«ЖАRA» «Чайный пьяница» «Рвы» GTA 4 (видеоигра...
122407,dataset/Dataset/data/970269/table_1,5,22 февраля 199328 ноября 1997[13]1 декабря 199...


### Ссылки википедии

In [9]:
import re

df_data_filter.loc[:, "column_data"] = df_data_filter["column_data"].progress_apply(lambda x: re.sub(r"\[[0-9]*\]", "", x))
df_data_filter

100%|█████████████████████████████| 1447089/1447089 [00:01<00:00, 992605.77it/s]


,table_id,column_id,column_data
0,dataset/Dataset/data/1000006/table_0,0,Театр им. Евгения Вахтангова (Москва) Театр им...
1,dataset/Dataset/data/1000007/table_1,2,"Кулагин и партнёры Люба, дети и завод… Врачебн..."
2,dataset/Dataset/data/1000023/table_0,0,1997 2002 2003 2003 2006—2007 2008 2010 2010 2...
3,dataset/Dataset/data/1000078/table_0,2,смешная жизни(Таисия)Приезжая Летняя поездка к...
4,dataset/Dataset/data/1000247/table_0,2,Семейные страсти В пасти безумия Гаррисон Берж...
...,...,...,...
122940,dataset/Dataset/data/999585/table_0,2,Трек-лист № Название Длительность 1. «Ангел м...
122941,dataset/Dataset/data/999746/table_3,20,НКЛ НКЛ
122942,dataset/Dataset/data/999874/table_0,5,5 из 9 8½ из 17 14 из 18 4½ из 15 14½ из 19 7 ...
122943,dataset/Dataset/data/999913/table_1,0,28-4-1 28-3-1 28-2-1 28-2 28-1 27-1 26-1 25-1 ...


In [10]:
empty_column_data_mask = df_data_filter["column_data"].apply(lambda x: str(x).replace(";", "").strip()) == ""

In [11]:
df_data_filter = df_data_filter[~empty_column_data_mask]

## JOIN заголовков с содержимым столбца

создание композитного ключа и merge(inner) двух датафреймов

### ColumnData dataframe

In [12]:
df_data_filter.loc[:, "composite_key"] = df_data_filter["table_id"] + " / " + df_data_filter["column_id"].astype(str)
df_data_filter

/tmp/ipykernel_19157/3755119166.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_filter.loc[:, "composite_key"] = df_data_filter["table_id"] + " / " + df_data_filter["column_id"].astype(str)


,table_id,column_id,column_data,composite_key
0,dataset/Dataset/data/1000006/table_0,0,Театр им. Евгения Вахтангова (Москва) Театр им...,dataset/Dataset/data/1000006/table_0 / 0
1,dataset/Dataset/data/1000007/table_1,2,"Кулагин и партнёры Люба, дети и завод… Врачебн...",dataset/Dataset/data/1000007/table_1 / 2
2,dataset/Dataset/data/1000023/table_0,0,1997 2002 2003 2003 2006—2007 2008 2010 2010 2...,dataset/Dataset/data/1000023/table_0 / 0
3,dataset/Dataset/data/1000078/table_0,2,смешная жизни(Таисия)Приезжая Летняя поездка к...,dataset/Dataset/data/1000078/table_0 / 2
4,dataset/Dataset/data/1000247/table_0,2,Семейные страсти В пасти безумия Гаррисон Берж...,dataset/Dataset/data/1000247/table_0 / 2
...,...,...,...,...
122940,dataset/Dataset/data/999585/table_0,2,Трек-лист № Название Длительность 1. «Ангел м...,dataset/Dataset/data/999585/table_0 / 2
122941,dataset/Dataset/data/999746/table_3,20,НКЛ НКЛ,dataset/Dataset/data/999746/table_3 / 20
122942,dataset/Dataset/data/999874/table_0,5,5 из 9 8½ из 17 14 из 18 4½ из 15 14½ из 19 7 ...,dataset/Dataset/data/999874/table_0 / 5
122943,dataset/Dataset/data/999913/table_1,0,28-4-1 28-3-1 28-2-1 28-2 28-1 27-1 26-1 25-1 ...,dataset/Dataset/data/999913/table_1 / 0


### Threshold dataframe

In [13]:
threshold = 0.78
df_threshold = pd.read_csv(f"../labelling/labelled/threshold_{threshold}.csv", sep=";")
df_threshold

,table_id,column_id,column_name_lemma,label,similarity
0,dataset/Dataset/data/1000006/table_0,0,театр,театр,1.000000
1,dataset/Dataset/data/1000006/table_0,1,спектакль,пьеса,0.789814
2,dataset/Dataset/data/1000006/table_0,4,примечание,примечание,1.000000
3,dataset/Dataset/data/1000006/table_1,0,год,год,1.000000
4,dataset/Dataset/data/1000006/table_1,2,название,название,1.000000
...,...,...,...,...,...
1504994,dataset/Dataset/data/999987/table_0,6,результат,результат,1.000000
1504995,dataset/Dataset/data/999987/table_0,2,турнир,турнир,1.000000
1504996,dataset/Dataset/data/999987/table_0,1,город,город,1.000000
1504997,dataset/Dataset/data/999987/table_0,0,год,год,1.000000


In [14]:
df_threshold.drop(["column_name_lemma", "similarity"], axis=1, inplace=True)
df_threshold

,table_id,column_id,label
0,dataset/Dataset/data/1000006/table_0,0,театр
1,dataset/Dataset/data/1000006/table_0,1,пьеса
2,dataset/Dataset/data/1000006/table_0,4,примечание
3,dataset/Dataset/data/1000006/table_1,0,год
4,dataset/Dataset/data/1000006/table_1,2,название
...,...,...,...
1504994,dataset/Dataset/data/999987/table_0,6,результат
1504995,dataset/Dataset/data/999987/table_0,2,турнир
1504996,dataset/Dataset/data/999987/table_0,1,город
1504997,dataset/Dataset/data/999987/table_0,0,год


In [15]:
df_threshold.loc[:, "composite_key"] = df_threshold["table_id"] + " / " + df_threshold["column_id"].astype(str)
df_threshold

,table_id,column_id,label,composite_key
0,dataset/Dataset/data/1000006/table_0,0,театр,dataset/Dataset/data/1000006/table_0 / 0
1,dataset/Dataset/data/1000006/table_0,1,пьеса,dataset/Dataset/data/1000006/table_0 / 1
2,dataset/Dataset/data/1000006/table_0,4,примечание,dataset/Dataset/data/1000006/table_0 / 4
3,dataset/Dataset/data/1000006/table_1,0,год,dataset/Dataset/data/1000006/table_1 / 0
4,dataset/Dataset/data/1000006/table_1,2,название,dataset/Dataset/data/1000006/table_1 / 2
...,...,...,...,...
1504994,dataset/Dataset/data/999987/table_0,6,результат,dataset/Dataset/data/999987/table_0 / 6
1504995,dataset/Dataset/data/999987/table_0,2,турнир,dataset/Dataset/data/999987/table_0 / 2
1504996,dataset/Dataset/data/999987/table_0,1,город,dataset/Dataset/data/999987/table_0 / 1
1504997,dataset/Dataset/data/999987/table_0,0,год,dataset/Dataset/data/999987/table_0 / 0


### Join dataframes

In [16]:
df_joined = df_threshold.merge(df_data_filter[["composite_key", "column_data"]], how="inner", on="composite_key")
df_joined.drop(["composite_key"], axis=1, inplace=True)
df_joined

,table_id,column_id,label,column_data
0,dataset/Dataset/data/1000006/table_0,0,театр,Театр им. Евгения Вахтангова (Москва) Театр им...
1,dataset/Dataset/data/1000006/table_0,1,пьеса,Дневник Анны ФранкФрэнсис Гудрич и Альберт Хак...
2,dataset/Dataset/data/1000006/table_0,4,примечание,видеофрагмент видеофрагмент телеспектакль виде...
3,dataset/Dataset/data/1000006/table_1,0,год,2004—2013 2006 2006 2006 2008 2007 2007 2008 2...
4,dataset/Dataset/data/1000006/table_1,2,название,"Кулагин и партнёры Люба, дети и завод… Врачебн..."
...,...,...,...,...
1439851,dataset/Dataset/data/999987/table_0,6,результат,7½ из 9 Нокаут-система 6½ из 9 8 из 11 6½ из 1...
1439852,dataset/Dataset/data/999987/table_0,2,турнир,71-й чемпионат Украины 73-й чемпионат Украины ...
1439853,dataset/Dataset/data/999987/table_0,1,город,Алушта Харьков Москва Киев Нью-Дели Киев Львов...
1439854,dataset/Dataset/data/999987/table_0,0,год,2002 2004 2010 2012 2012 2013 2014 2016 2018


### Create label_id

In [17]:
labels = list(df_joined["label"].unique())
labels.sort()

sem_types = {v: i for i, v in enumerate(labels)}
sem_types

{'авиакомпания': 0,
 'автомобиль': 1,
 'адрес': 2,
 'актер': 3,
 'альбом': 4,
 'аннотация': 5,
 'арена': 6,
 'атлет': 7,
 'аэропорт': 8,
 'база': 9,
 'банк': 10,
 'боксер': 11,
 'борец': 12,
 'ввп': 13,
 'ведущий': 14,
 'вес': 15,
 'вещество': 16,
 'владелец': 17,
 'возраст': 18,
 'высота': 19,
 'гимнаст': 20,
 'глубина': 21,
 'год': 22,
 'гонка': 23,
 'город': 24,
 'группа': 25,
 'дата': 26,
 'двигатель': 27,
 'день': 28,
 'директор': 29,
 'документ': 30,
 'дорога': 31,
 'жанр': 32,
 'животное': 33,
 'журнал': 34,
 'закон': 35,
 'звук': 36,
 'здание': 37,
 'игра': 38,
 'идентификатор': 39,
 'идеология': 40,
 'издатель': 41,
 'изображение': 42,
 'инструмент': 43,
 'камера': 44,
 'канал': 45,
 'категория': 46,
 'класс': 47,
 'классификация': 48,
 'клуб': 49,
 'книга': 50,
 'код': 51,
 'количество': 52,
 'колледж': 53,
 'команда': 54,
 'компания': 55,
 'компонент': 56,
 'континент': 57,
 'конькобежец': 58,
 'корабль': 59,
 'кратер': 60,
 'лейбл': 61,
 'лес': 62,
 'лига': 63,
 'локомотив'

In [18]:
df_joined["label_id"] = df_joined["label"].progress_apply(lambda x: sem_types[x])
df_joined

100%|████████████████████████████| 1439856/1439856 [00:00<00:00, 2295547.82it/s]


,table_id,column_id,label,column_data,label_id
0,dataset/Dataset/data/1000006/table_0,0,театр,Театр им. Евгения Вахтангова (Москва) Театр им...,145
1,dataset/Dataset/data/1000006/table_0,1,пьеса,Дневник Анны ФранкФрэнсис Гудрич и Альберт Хак...,110
2,dataset/Dataset/data/1000006/table_0,4,примечание,видеофрагмент видеофрагмент телеспектакль виде...,101
3,dataset/Dataset/data/1000006/table_1,0,год,2004—2013 2006 2006 2006 2008 2007 2007 2008 2...,22
4,dataset/Dataset/data/1000006/table_1,2,название,"Кулагин и партнёры Люба, дети и завод… Врачебн...",73
...,...,...,...,...,...
1439851,dataset/Dataset/data/999987/table_0,6,результат,7½ из 9 Нокаут-система 6½ из 9 8 из 11 6½ из 1...,118
1439852,dataset/Dataset/data/999987/table_0,2,турнир,71-й чемпионат Украины 73-й чемпионат Украины ...,154
1439853,dataset/Dataset/data/999987/table_0,1,город,Алушта Харьков Москва Киев Нью-Дели Киев Львов...,24
1439854,dataset/Dataset/data/999987/table_0,0,год,2002 2004 2010 2012 2012 2013 2014 2016 2018,22


### Reorder columns

In [19]:
df_joined = df_joined[["table_id", "column_id", "label_id", "label", "column_data"]]
df_joined

,table_id,column_id,label_id,label,column_data
0,dataset/Dataset/data/1000006/table_0,0,145,театр,Театр им. Евгения Вахтангова (Москва) Театр им...
1,dataset/Dataset/data/1000006/table_0,1,110,пьеса,Дневник Анны ФранкФрэнсис Гудрич и Альберт Хак...
2,dataset/Dataset/data/1000006/table_0,4,101,примечание,видеофрагмент видеофрагмент телеспектакль виде...
3,dataset/Dataset/data/1000006/table_1,0,22,год,2004—2013 2006 2006 2006 2008 2007 2007 2008 2...
4,dataset/Dataset/data/1000006/table_1,2,73,название,"Кулагин и партнёры Люба, дети и завод… Врачебн..."
...,...,...,...,...,...
1439851,dataset/Dataset/data/999987/table_0,6,118,результат,7½ из 9 Нокаут-система 6½ из 9 8 из 11 6½ из 1...
1439852,dataset/Dataset/data/999987/table_0,2,154,турнир,71-й чемпионат Украины 73-й чемпионат Украины ...
1439853,dataset/Dataset/data/999987/table_0,1,24,город,Алушта Харьков Москва Киев Нью-Дели Киев Львов...
1439854,dataset/Dataset/data/999987/table_0,0,22,год,2002 2004 2010 2012 2012 2013 2014 2016 2018


In [20]:
df_joined[df_joined["table_id"] == "dataset/Dataset/data/1367439/table_0"]

,table_id,column_id,label_id,label,column_data
91760,dataset/Dataset/data/1367439/table_0,1,95,пол,неизвестен


In [21]:
column_data_na = df_joined["column_data"].isna().sum()
column_data_na
# df_joined = df_joined[~column_data_na]

0

In [22]:
df_joined

,table_id,column_id,label_id,label,column_data
0,dataset/Dataset/data/1000006/table_0,0,145,театр,Театр им. Евгения Вахтангова (Москва) Театр им...
1,dataset/Dataset/data/1000006/table_0,1,110,пьеса,Дневник Анны ФранкФрэнсис Гудрич и Альберт Хак...
2,dataset/Dataset/data/1000006/table_0,4,101,примечание,видеофрагмент видеофрагмент телеспектакль виде...
3,dataset/Dataset/data/1000006/table_1,0,22,год,2004—2013 2006 2006 2006 2008 2007 2007 2008 2...
4,dataset/Dataset/data/1000006/table_1,2,73,название,"Кулагин и партнёры Люба, дети и завод… Врачебн..."
...,...,...,...,...,...
1439851,dataset/Dataset/data/999987/table_0,6,118,результат,7½ из 9 Нокаут-система 6½ из 9 8 из 11 6½ из 1...
1439852,dataset/Dataset/data/999987/table_0,2,154,турнир,71-й чемпионат Украины 73-й чемпионат Украины ...
1439853,dataset/Dataset/data/999987/table_0,1,24,город,Алушта Харьков Москва Киев Нью-Дели Киев Львов...
1439854,dataset/Dataset/data/999987/table_0,0,22,год,2002 2004 2010 2012 2012 2013 2014 2016 2018


### Remove stupid .mw-parser ._.

In [23]:
# df_joined.loc[:, "column_data"] = df_joined["column_data"].progress_apply(lambda x: re.sub(r"\.mw-parser-output \.ts.+{.+}", "", x))
df_joined["column_data"].str.contains("mw-parser").sum()

0

In [24]:
df_joined[df_joined["table_id"] == "dataset/Dataset/data/1012855/table_1"]

,table_id,column_id,label_id,label,column_data
2303,dataset/Dataset/data/1012855/table_1,5,26,дата,22 февраля 199328 ноября 19971 декабря 19974 я...
2304,dataset/Dataset/data/1012855/table_1,1,73,название,«Вавилон-5: Сборы» «Babylon 5: The Gathering»
2305,dataset/Dataset/data/1012855/table_1,3,144,сценарист,Дж. Майкл Стражински
2306,dataset/Dataset/data/1012855/table_1,2,3,актер,Ричард Комптон
2307,dataset/Dataset/data/1012855/table_1,7,51,код,0
2308,dataset/Dataset/data/1012855/table_1,4,106,продюсер,Роберт Латман Браун


In [25]:
# tokenizer will take care of it
# df_joined.loc[:, "column_data"] = df_joined["column_data"].progress_apply(lambda x: str(x).lower())

### Save dataset

In [26]:
df_joined.to_csv("./dataset/cta_dataset.csv", sep="|", index=False)

In [27]:
# import numpy as np

# number_of_chunks = 12
# for id, chunk in enumerate(np.array_split(df_joined, number_of_chunks)):
#     chunk.to_csv(f"./data/data_{id}.csv", sep="|", index=False)